<a href="https://colab.research.google.com/github/rcelebi/iswc2020-smarttask/blob/master/TFIDF_type_pred_dbpedia_frequent_type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
import os
path_to_ipynb = '/content/drive/My Drive/iswc-ammar/shervin'
# change directory to where autoregq.py, src, and data are
os.chdir(path_to_ipynb)

In [6]:
import pandas as pd
import numpy as np
dbpedia_df = pd.read_json('data/smarttask_dbpedia_train.json')

In [7]:
test_df= pd.read_json('data/smarttask_dbpedia_test.json')

In [8]:
test_df.head()

,id,question,category,type
0,dbpedia_16015,How many ingredients are in the grain} ?,literal,[number]
1,dbpedia_3885,Is the case fatality rate of Fournier gangrene...,boolean,[boolean]
2,dbpedia_12907,Does the shelf life of spinach equal 8?,boolean,[boolean]
3,dbpedia_7955,What sound does a pig make in the French langu...,literal,[string]
4,dbpedia_2376,When was Fergie completed his record label in ...,literal,[date]


In [9]:
answer_types = {}
for answers in dbpedia_df.type:
    for a in answers:
        if a in answer_types:
            answer_types[a]+=1
        else:
            answer_types[a]=1

def multitype_to_freqtype(type_list):
    for t in sorted(sorted_types, key=lambda item: item[1]):
        if sorted_types[t] < 1000: 
          if t in type_list:
              return t

    for t in sorted(sorted_types, key=lambda item: item[1]):
          if t in type_list:
              return t
    

def types_to_bottom_type(type_list):
    if len(type_list) == 0:
      return None 
    return type_list[0]


def types_to_top_type(type_list):
    if len(type_list) == 0:
      return None 
    return type_list[-1]
            
answ_df =pd.DataFrame.from_dict(answer_types, orient='index', columns=['Freq'])
sorted_types = answ_df.sort_values(by='Freq', ascending=False).to_dict()['Freq']

dbpedia_df['frequent_type'] =dbpedia_df.type.apply(multitype_to_freqtype)
dbpedia_df['bottom_level_type'] =dbpedia_df.type.apply(types_to_bottom_type)
dbpedia_df['top_level_type'] =dbpedia_df.type.apply(types_to_top_type)

In [10]:
test_df['frequent_type'] =test_df.type.apply(multitype_to_freqtype)
test_df['bottom_level_type'] =test_df.type.apply(types_to_bottom_type)
test_df['top_level_type'] =test_df.type.apply(types_to_top_type)

In [11]:
dbpedia_df= dbpedia_df[dbpedia_df.category.notna()]
test_df= test_df[test_df.category.notna()]

In [12]:
dbpedia_df= dbpedia_df[dbpedia_df['frequent_type'].notna()]
test_df= test_df[test_df['frequent_type'].notna()]

In [13]:
dbpedia_df.dropna( subset=['question'], inplace=True)
test_df.dropna( subset=['question'], inplace=True)

In [14]:
dbpedia_df.head()

,id,question,category,type,frequent_type,bottom_level_type,top_level_type
0,dbpedia_1177,Was Jacqueline Kennedy Onassis a follower of M...,boolean,[boolean],boolean,boolean,boolean
1,dbpedia_14427,What is the name of the opera based on Twelfth...,resource,"[dbo:Opera, dbo:MusicalWork, dbo:Work]",dbo:Work,dbo:Opera,dbo:Work
2,dbpedia_16615,When did Lena Horne receive the Grammy Award f...,literal,[date],date,date,date
3,dbpedia_23480,Do Prince Harry and Prince William have the sa...,boolean,[boolean],boolean,boolean,boolean
4,dbpedia_3681,What is the subsidiary company working for Leo...,resource,"[dbo:EducationalInstitution, dbo:Organisation,...",dbo:EducationalInstitution,dbo:EducationalInstitution,dbo:Agent


In [15]:
def extend_cat(row):
  if row['category'] =='boolean':
    return row['category']
  elif row['category'] == 'literal':
    return row['frequent_type']
  else:
    return row['category']


In [16]:
dbpedia_df['extended_cat'] = dbpedia_df.apply(extend_cat, axis=1)

In [17]:
dbpedia_res_df = dbpedia_df[dbpedia_df.category=='resource']

In [18]:
VALIDATION_SPLIT = 0.2
indicies = np.arange(dbpedia_res_df.shape[0])
np.random.shuffle(indicies)
num_valid_samples = int(VALIDATION_SPLIT * dbpedia_res_df.shape[0])
num_valid_samples

1911

In [19]:
num_test_samples=  int(num_valid_samples/2)

In [20]:
dbpedia_res_df=dbpedia_res_df.iloc[indicies]

In [21]:
train_df= dbpedia_res_df[:-num_valid_samples]
train_df.head()

,id,question,category,type,frequent_type,bottom_level_type,top_level_type,extended_cat
15557,dbpedia_11753,Where does Thomas Jefferson's special people s...,resource,"[dbo:Building, dbo:ArchitecturalStructure, dbo...",dbo:ArchitecturalStructure,dbo:Building,dbo:Location,resource
15040,dbpedia_8745,Mention the federative entity of Mexico respon...,resource,"[dbo:City, dbo:Settlement, dbo:PopulatedPlace,...",dbo:Settlement,dbo:City,dbo:Location,resource
11744,dbpedia_10590,Who is the contestant of part of the series by...,resource,"[dbo:Person, dbo:Agent]",dbo:Agent,dbo:Person,dbo:Agent,resource
10080,dbpedia_22442,Which sports are played in schools affiliated ...,resource,"[dbo:Sport, dbo:Activity]",dbo:Activity,dbo:Sport,dbo:Activity,resource
9229,dbpedia_6107,Tell me studio album whose name has the word w...,resource,"[dbo:Album, dbo:MusicalWork, dbo:Work]",dbo:Work,dbo:Album,dbo:Work,resource


In [22]:
valid_df= dbpedia_res_df[-num_valid_samples:]
valid_df.head()

,id,question,category,type,frequent_type,bottom_level_type,top_level_type,extended_cat
1117,dbpedia_18221,Which company's division is Audi India? Correc...,resource,"[dbo:Company, dbo:Organisation, dbo:Agent]",dbo:Company,dbo:Company,dbo:Agent,resource
3803,dbpedia_23544,Who is the president of Eritrea?,resource,"[dbo:OfficeHolder, dbo:Person, dbo:Agent]",dbo:OfficeHolder,dbo:OfficeHolder,dbo:Agent,resource
5912,dbpedia_13588,What is the wiki article for a list of fairy t...,resource,[dbo:Media],dbo:Media,dbo:Media,dbo:Media,resource
1528,dbpedia_5423,Which uncle did a tribute to Christ Church?,resource,"[dbo:Person, dbo:Agent]",dbo:Agent,dbo:Person,dbo:Agent,resource
16019,dbpedia_1725,Where is Glasgow nearest body of water located...,resource,"[dbo:River, dbo:Stream, dbo:BodyOfWater, dbo:N...",dbo:NaturalPlace,dbo:River,dbo:Location,resource


In [23]:
len(valid_df)

1911

In [24]:
#valid_df.to_csv('/content/drive/My Drive/fastai-v3/data/smarttask/dbpedia_valid_gold.csv', index=False)

In [25]:
#test_df= dbpedia_df[-num_test_samples:]
#test_df.head()

In [26]:
len(valid_df)

1911

## Most frequent/generic type prediction

In [27]:
target = 'frequent_type'


In [28]:
train_res_df= train_df[train_df.category =='resource']
valid_res_df= valid_df[valid_df.category =='resource']
test_res_df= test_df[test_df.category =='resource']
len(train_res_df),len(valid_res_df),len(test_res_df)

(7646, 1911, 2445)

In [29]:
all_res_df = train_res_df.append(valid_res_df).append(test_res_df) 
n_utypes = all_res_df[target].unique()
print (len(n_utypes))

88


In [30]:
len(train_res_df), len(valid_res_df)

(7646, 1911)

In [31]:
type_hierarchy = {}
to_upper_class = {}
for i, row in dbpedia_res_df.type.iteritems():
    #print (row)
    types= list(row)
    for i in range(len(types)-1):
      parent = types[i]
      child = types[i+1]
      #print (parent, child) 
      if parent not in type_hierarchy:
          type_hierarchy[parent] = [child]
      else:
          type_hierarchy[parent].append(child)
          
      if child not in to_upper_class:
          to_upper_class[child] = parent

In [32]:
def to_super_class(specific_type, type_with_less_freq):
    if specific_type in type_with_less_freq:
        if specific_type in to_upper_class:
            #if to_upper_class[specific_type] == 'owl:Thing':
            #   return specific_type
            return to_upper_class[specific_type]
    return specific_type

def to_sub_class(specific_type, type_with_less_freq):
    if specific_type in type_with_less_freq:
        if specific_type in to_upper_class:
            #if to_upper_class[specific_type] == 'owl:Thing':
            #   return specific_type
            return type_hierarchy[parent]
    return specific_type

def choose_freqent_types_balancing(specific_type, iters, min_freq):
    for i in range(iters):
        specific_type_freq = specific_type.value_counts()
        type_with_less_freq = list(specific_type_freq[specific_type_freq < min_freq].keys())
        specific_type = specific_type.apply(lambda x: to_super_class(x, type_with_less_freq) )
    return specific_type

def balance_types(specific_type, min_freq, max_freq):
    # slipt to super-classes if number of 
    
    specific_type_freq = specific_type.value_counts()
    max_freq =  specific_type_freq.sum() * 2  / len(specific_type_freq)
    type_with_most_freq = list(specific_type_freq[specific_type_freq > max_freq].keys())
    type_with_less_freq = list(specific_type_freq[specific_type_freq < min_freq].keys())
    
    specific_type = specific_type.apply(lambda x: to_super_class(x, type_with_less_freq) )


def choose_freqent_types_balancing(specific_type, iters, min_freq):
    for i in range(iters):
        specific_type_freq = specific_type.value_counts()
        type_with_less_freq = list(specific_type_freq[specific_type_freq < min_freq].keys())
        specific_type = specific_type.apply(lambda x: to_super_class(x, type_with_less_freq) )
    return specific_type  

In [33]:
#train_res_df['frequent_type'] =choose_freqent_types_balancing(train_res_df['frequent_type'], 50, 200)

### TF-IDF Text Classifier


In [34]:
label_dict = {}
for index, possible_label in enumerate(sorted(all_res_df[target].unique())):
    label_dict[possible_label] = index
label_dict


{'dbo:Activity': 0,
 'dbo:AdministrativeRegion': 1,
 'dbo:Agent': 2,
 'dbo:AnatomicalStructure': 3,
 'dbo:Architect': 4,
 'dbo:ArchitecturalStructure': 5,
 'dbo:Artist': 6,
 'dbo:Astronaut': 7,
 'dbo:Athlete': 8,
 'dbo:Award': 9,
 'dbo:Band': 10,
 'dbo:BaseballTeam': 11,
 'dbo:BeautyQueen': 12,
 'dbo:Biomolecule': 13,
 'dbo:Broadcaster': 14,
 'dbo:Case': 15,
 'dbo:CelestialBody': 16,
 'dbo:Cemetery': 17,
 'dbo:ChemicalSubstance': 18,
 'dbo:City': 19,
 'dbo:Cleric': 20,
 'dbo:CollegeCoach': 21,
 'dbo:Company': 22,
 'dbo:Country': 23,
 'dbo:Currency': 24,
 'dbo:Deity': 25,
 'dbo:Device': 26,
 'dbo:Disease': 27,
 'dbo:EducationalInstitution': 28,
 'dbo:EthnicGroup': 29,
 'dbo:Event': 30,
 'dbo:Family': 31,
 'dbo:FictionalCharacter': 32,
 'dbo:FigureSkater': 33,
 'dbo:Flag': 34,
 'dbo:Food': 35,
 'dbo:GovernmentAgency': 36,
 'dbo:Holiday': 37,
 'dbo:HorseRace': 38,
 'dbo:Island': 39,
 'dbo:Language': 40,
 'dbo:Legislature': 41,
 'dbo:MeanOfTransportation': 42,
 'dbo:Media': 43,
 'dbo:Medic

In [37]:
labels_train = train_res_df[target].replace(label_dict).values

labels_val = valid_res_df[target].replace(label_dict).values


In [35]:
labels_test = test_res_df[target].replace(label_dict).values

In [36]:
from sklearn import tree, ensemble
from sklearn.pipeline import Pipeline

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.tree import DecisionTreeClassifier


In [39]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', DecisionTreeClassifier()),
])

In [40]:
pipeline.fit(train_res_df['question'], labels_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
              

In [43]:
pipeline.score(valid_res_df['question'], labels_val)

0.47514390371533227

In [44]:
pipeline.score(test_res_df['question'], labels_test)

0.4560327198364008

## RandomForest Classifier

In [49]:
pipeline1 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', ensemble.RandomForestClassifier(n_estimators=200)),
])

In [50]:
pipeline1.fit(train_res_df['question'], labels_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

In [51]:
pipeline1.score(valid_res_df['question'], labels_val)

0.564625850340136

In [52]:
pipeline1.score(test_res_df['question'], labels_test)

0.5488752556237219

## SVM Classifier

In [53]:
from sklearn.svm import LinearSVC

In [54]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC()),
])

In [56]:
pipeline2.fit(train_res_df['question'], labels_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                     

In [57]:
pipeline2.score(valid_res_df['question'], labels_val)

0.6755625327053898

In [58]:
pipeline2.score(test_res_df['question'], labels_test)

0.6768916155419223